In [27]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,\
RandomizedSearchCV

In [2]:
df = pd.read_csv("final.csv",low_memory =False)

In [3]:
df = df.drop(['Timestamp', 'X Value', 'Y Value', 'Z Value'] + [col for col in df.columns if 'Quality' in col], axis=1)

In [4]:
df.shape

(201000, 14)

In [5]:
# Step 1: Data Cleaning
# Handle missing or corrupted values

# Replace missing values with the mean of the column
df = df.replace("-", np.nan)
df = df.fillna(df.mean(numeric_only=True))

df.drop_duplicates(inplace=True)
df = df.dropna()

In [6]:
min_max_scaler = preprocessing.MinMaxScaler()
df_scaled = min_max_scaler.fit_transform(df)
df = pd.DataFrame(df_scaled, columns=df.columns)

In [7]:
X = df.iloc[:,:-1]
X

,F3 Value,FC5 Value,F7 Value,T7 Value,P7 Value,O1 Value,O2 Value,P8 Value,T8 Value,F8 Value,AF4 Value,FC6 Value,F4 Value
0,0.159986,0.812773,0.213348,0.665521,0.523002,0.519784,0.580703,0.617303,0.576717,0.533732,0.652132,0.599745,0.616328
1,0.159761,0.811747,0.215832,0.675258,0.516214,0.518930,0.584828,0.618023,0.579399,0.530740,0.655246,0.600806,0.618421
2,0.158635,0.810977,0.212993,0.674685,0.523379,0.519214,0.583393,0.618203,0.579310,0.531556,0.656025,0.597517,0.617225
3,0.157284,0.810977,0.214058,0.674971,0.525641,0.518930,0.578730,0.616493,0.579220,0.529108,0.651742,0.598366,0.615132
4,0.157284,0.813029,0.213703,0.674971,0.516968,0.518930,0.580344,0.616042,0.579667,0.529924,0.652521,0.604733,0.616328
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180982,0.157622,0.813798,0.213348,0.665521,0.516968,0.521492,0.565638,0.587144,0.564109,0.529924,0.631302,0.577674,0.589414
180983,0.158973,0.817133,0.214058,0.668385,0.525641,0.526615,0.569225,0.590025,0.569653,0.531012,0.629356,0.577037,0.590311
180984,0.157960,0.815594,0.211928,0.668958,0.524133,0.523200,0.568508,0.588945,0.563394,0.532372,0.628188,0.578417,0.590012
180985,0.158185,0.812773,0.212283,0.667239,0.520362,0.520638,0.567073,0.587414,0.560354,0.531828,0.629161,0.580115,0.590909


In [8]:
y = df.iloc[:,-1]
y

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
180982    0.0
180983    0.0
180984    0.0
180985    0.0
180986    0.0
Name: truth, Length: 180987, dtype: float64

## Random Forest

In [64]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=150, random_state=42,max_features='sqrt',max_depth=75)
rf.fit(X_train, y_train)

# Predict on the test set and compute accuracy
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))


Accuracy: 90.06%


In [45]:
param_grid = {
    'n_estimators': [25],
    'max_features': ['sqrt',None],
    'max_depth': [9,10],
    'max_leaf_nodes': [9],
}

In [46]:
grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

RandomForestClassifier(max_depth=9, max_features='sqrt', max_leaf_nodes=9,
                       n_estimators=25)


In [68]:
# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 score: {:.2f}".format(f1))

Accuracy: 0.9006
Precision: 0.89
Recall: 0.94
F1 score: 0.91


In [69]:

# print the scores on training and test set
print('Training set score: {:.4f}'.format(rf.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(rf.score(X_test, y_test)))

Training set score: 1.0000
Test set score: 0.9006


In [70]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score

# Define the scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score)}

# Perform cross-validation
cv_results = cross_validate(rf, X, y, cv=5, scoring=scoring)

# Print the results
print("Mean accuracy:", cv_results['test_accuracy'].mean())
print("Mean precision:", cv_results['test_precision'].mean())
print("Mean recall:", cv_results['test_recall'].mean())


Mean accuracy: 0.4735092258545407
Mean precision: 0.5176539922718089
Mean recall: 0.5396554571601742


## Decision Tree Classifier

In [13]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

dtc.fit(X_train, y_train)

# use the trained classifier to predict the labels of the test data
y_pred = dtc.predict(X_test)

# calculate the accuracy, precision, and recall of the classifier
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)

Accuracy: 0.783689706613625
Precision: 0.8032877261564677
Recall: 0.8040937139417047


In [14]:

# print the scores on training and test set
print('Training set score: {:.4f}'.format(dtc.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(dtc.score(X_test, y_test)))

Training set score: 1.0000
Test set score: 0.7837


In [71]:
# Define the scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score)}

# Perform cross-validation
cv_results = cross_validate(dtc, X, y, cv=5, scoring=scoring)

# Print the results
print("Mean accuracy:", cv_results['test_accuracy'].mean())
print("Mean precision:", cv_results['test_precision'].mean())
print("Mean recall:", cv_results['test_recall'].mean())

Mean accuracy: 0.4744153438011171
Mean precision: 0.5235292695788871
Mean recall: 0.498335011793824


## Gradient Boosting

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

clf = GradientBoostingClassifier(n_estimators=100, random_state=50)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# calculate the accuracy, precision, and recall of the classifier
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)

Accuracy: 0.639234211834908
Precision: 0.6252094717668488
Recall: 0.8609842973962776


In [16]:

# print the scores on training and test set
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

Training set score: 0.6410
Test set score: 0.6392


In [72]:
# Define the scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score)}

# Perform cross-validation
cv_results = cross_validate(clf, X, y, cv=5, scoring=scoring)

# Print the results
print("Mean accuracy:", cv_results['test_accuracy'].mean())
print("Mean precision:", cv_results['test_precision'].mean())
print("Mean recall:", cv_results['test_recall'].mean())

Mean accuracy: 0.49412965759840005
Mean precision: 0.5296334601651183
Mean recall: 0.6768587833661792


## Gaussian NB

In [17]:
from sklearn.naive_bayes import GaussianNB

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

# instantiate the model
gnb = GaussianNB()


# fit the model
gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)
# calculate the accuracy, precision, and recall of the classifier
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)

Accuracy: 0.4795568815956683
Precision: 0.5827283726557774
Recall: 0.19329754678171876


In [18]:

# print the scores on training and test set
print('Training set score: {:.4f}'.format(gnb.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(gnb.score(X_test, y_test)))

Training set score: 0.4810
Test set score: 0.4796


In [73]:
# Define the scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score)}

# Perform cross-validation
cv_results = cross_validate(gnb, X, y, cv=5, scoring=scoring)

# Print the results
print("Mean accuracy:", cv_results['test_accuracy'].mean())
print("Mean precision:", cv_results['test_precision'].mean())
print("Mean recall:", cv_results['test_recall'].mean())

Mean accuracy: 0.44408153497553593
Mean precision: 0.5014694217879845
Mean recall: 0.19539933491223294


## Save Model

In [74]:
import pickle
filename = 'finalized_EEG_model.sav'
pickle.dump(rf, open(filename, 'wb'))

In [79]:
loaded_model = pickle.load(open(filename, 'rb'))
Y_test = y_test
X_test = []
result = loaded_model.predict()
print(result)

0.9006022432178573


In [106]:
ch = pd.read_csv('tru.csv')
ch = ch.iloc[0:1]


In [107]:

ch

,Timestamp,F3 Value,F3 Quality,FC5 Value,5C5 Quality,F7 Value,F7 Quality,T7 Value,T7 Quality,P7 Value,...,AF4 Value,AF4 Quality,FC6 Value,FC6 Quality,F4 Value,F4 Quality,X Value,Y Value,Z Value,truth
0,2018-08-16 18:47:16.402609,-107.0,0.0,-118.0,0.0,-122.0,0.0,-131.0,0.0,-113.0,...,-116.0,0.0,-93.0,0.0,-82.0,0.0,22.0,24.0,?,1


In [109]:
ch = ch.drop([col for col in ch if 'Quality' in col],axis=1)

In [112]:
ch = ch.drop(['Timestamp', 'X Value', 'Y Value', 'Z Value'],axis=1)
ch

,F3 Value,FC5 Value,F7 Value,T7 Value,P7 Value,O1 Value,O2 Value,P8 Value,T8 Value,F8 Value,AF4 Value,FC6 Value,F4 Value,truth
0,-107.0,-118.0,-122.0,-131.0,-113.0,-121.0,-109.0,-90.0,-127.0,-110.0,-116.0,-93.0,-82.0,1


In [131]:
X = ch.iloc[:,:-1]
X

,F3 Value,FC5 Value,F7 Value,T7 Value,P7 Value,O1 Value,O2 Value,P8 Value,T8 Value,F8 Value,AF4 Value,FC6 Value,F4 Value
0,-107.0,-118.0,-122.0,-131.0,-113.0,-121.0,-109.0,-90.0,-127.0,-110.0,-116.0,-93.0,-82.0


In [132]:
y = ch.iloc[:,-1]
y

0    1
Name: truth, dtype: int64

In [134]:
min_max_scaler = preprocessing.MinMaxScaler()
c = min_max_scaler.fit_transform(X)
c

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [133]:
result = loaded_model.predict(X)
print(result)

[0.]


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
